In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_chuko_mansions():
    base_url = 'https://www.homes.co.jp/mansion/chuko/tokyo/23ku/list/'
    max_page = 188  # 最大ページ数
    mansions = []

    for page in range(1, max_page):
        url = f'{base_url}?page={page}'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        for mansion in soup.select('.moduleInner'):
            try:
                name = mansion.select_one('.bukkenName').text.strip()

                # 所在地を取得
                location_parts = mansion.select('td')[0].text.strip()

                # 価格を取得
                price_text = mansion.select_one('.priceLabel').text.strip()
                price = int(price_text.replace('万円', '').replace(',', '')) * 10000

                # 間取りと専有面積を分けて取得
                layout_and_area = mansion.select_one('.layoutSpace').text.strip()
                layout, area = layout_and_area.split('/')

                # 築年数を取得
                construction_year_text = mansion.select('td')[2].text.strip()
                construction_year = construction_year_text.split('（')[1].split('）')[0]

                mansions.append({
                    'name': name,
                    'location_parts': location_parts,
                    'price': price,
                    'layout': layout.strip(),
                    'area': area.strip(),
                    'construction_year': construction_year
                })
            except AttributeError:
                continue

    df = pd.DataFrame(mansions)
    return df

# マンション情報を取得
mansion_data = scrape_chuko_mansions()
print(mansion_data)


                  name                         location_parts      price  \
0            カーサフェリス洗足         東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16   79900000   
1     ライオンズマンション西麻布シティ     東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8  113900000   
2          イニシアイオ新宿夏目坂       都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3   95800000   
3          クレストコート杉並宮前     京王井の頭線 富士見ヶ丘駅 徒歩11分東京都杉並区宮前1丁目15-1   74800000   
4         パークハウス高円寺南桃園  東京メトロ丸ノ内線 東高円寺駅 徒歩8分東京都杉並区高円寺南5丁目8-12   82900000   
...                ...                                    ...        ...   
5605            アウルタワー        東京メトロ有楽町線 東池袋駅 徒歩2分東京都豊島区東池袋4丁目  218000000   
5606    JR山手線 渋谷駅 徒歩8分                JR山手線 渋谷駅 徒歩8分東京都渋谷区鶯谷町  298000000   
5607   JR山手線 目白駅 徒歩11分            西武新宿線 下落合駅 徒歩7分東京都新宿区下落合4丁目  208000000   
5608   大崎ウエストシティタワーズW棟              JR山手線 大崎駅 徒歩4分東京都品川区大崎2丁目  229800000   
5609        中目黒アトラスタワー    東京メトロ日比谷線 中目黒駅 徒歩2分東京都目黒区上目黒1丁目26-1  328000000   

     layout      area construction_year  
0      3LDK   60.32m²              築22年  
1  

In [13]:
import re

# 正規表現を使用して情報を抽出
mansion_data["transportation"] = mansion_data["location_parts"].apply(lambda x: re.search(r"(.*) 徒歩(\d+)分", x).group(1) if isinstance(x, str) else None)
mansion_data["minutes"] = mansion_data["location_parts"].apply(lambda x: re.search(r"徒歩(\d+)分", x).group(1) if isinstance(x, str) else None)
mansion_data["address"] = mansion_data["location_parts"].apply(lambda x: re.search(r"徒歩\d+分(.*)", x).group(1) if isinstance(x, str) else None)

# 「m²」を削除して数値に変換
mansion_data['area'] = mansion_data['area'].apply(lambda x: float(x.replace('m²', '')) if isinstance(x, str) else None)

# 所在地から「東京都」の後から「○○区」までを抜き出す関数
def extract_district(address):
    match = re.search(r'東京都(.+?区)', address)
    if match:
        return match.group(1)
    else:
        return None

# 区という項目に出力
mansion_data['区'] = mansion_data['address'].apply(extract_district)

# 正規表現パターン（キャプチャグループを追加）
pattern = r"築(\d+)年"

# 築年数から数字のみを抽出して修正
mansion_data['construction_year'] = mansion_data['construction_year'].str.extract(pattern).astype(int)

mansion_data

,name,location_parts,price,layout,area,construction_year,transportation,minutes,address,区
0,カーサフェリス洗足,東急目黒線 洗足駅 徒歩10分東京都品川区荏原7丁目9-16,79900000,3LDK,None,22,東急目黒線 洗足駅,10,東京都品川区荏原7丁目9-16,品川区
1,ライオンズマンション西麻布シティ,東京メトロ日比谷線 六本木駅 徒歩9分東京都港区西麻布2丁目13-8,113900000,2LDK,None,25,東京メトロ日比谷線 六本木駅,9,東京都港区西麻布2丁目13-8,港区
2,イニシアイオ新宿夏目坂,都営大江戸線 若松河田駅 徒歩6分東京都新宿区原町3丁目64-3,95800000,2LDK,None,15,都営大江戸線 若松河田駅,6,東京都新宿区原町3丁目64-3,新宿区
3,クレストコート杉並宮前,京王井の頭線 富士見ヶ丘駅 徒歩11分東京都杉並区宮前1丁目15-1,74800000,3LDK,None,28,京王井の頭線 富士見ヶ丘駅,11,東京都杉並区宮前1丁目15-1,杉並区
4,パークハウス高円寺南桃園,東京メトロ丸ノ内線 東高円寺駅 徒歩8分東京都杉並区高円寺南5丁目8-12,82900000,3LDK,None,23,東京メトロ丸ノ内線 東高円寺駅,8,東京都杉並区高円寺南5丁目8-12,杉並区
...,...,...,...,...,...,...,...,...,...,...
5605,アウルタワー,東京メトロ有楽町線 東池袋駅 徒歩2分東京都豊島区東池袋4丁目,218000000,2LDK,None,14,東京メトロ有楽町線 東池袋駅,2,東京都豊島区東池袋4丁目,豊島区
5606,JR山手線 渋谷駅 徒歩8分,JR山手線 渋谷駅 徒歩8分東京都渋谷区鶯谷町,298000000,2LDK,None,13,JR山手線 渋谷駅,8,東京都渋谷区鶯谷町,渋谷区
5607,JR山手線 目白駅 徒歩11分,西武新宿線 下落合駅 徒歩7分東京都新宿区下落合4丁目,208000000,3LDK,None,22,西武新宿線 下落合駅,7,東京都新宿区下落合4丁目,新宿区
5608,大崎ウエストシティタワーズW棟,JR山手線 大崎駅 徒歩4分東京都品川区大崎2丁目,229800000,2SLDK,None,15,JR山手線 大崎駅,4,東京都品川区大崎2丁目,品川区


In [31]:
#import gspread
#from gspread_dataframe import set_with_dataframe

# Google Sheets の認証
#gc = gspread.service_account(filename='spread-sheet-api-419202-59ee0bee5075.json')  # 認証情報ファイルのパスを指定

# スプレッドシートを開く（スプレッドシートの URL から ID を取得して指定）
#spreadsheet_id = '1qjYieNxp0gG3fOuDNkV46DL4Q1wK8vn_kcZTiYwtVIw'
#worksheet_name = '購入物件'
#sh = gc.open_by_key(spreadsheet_id)
#worksheet = sh.worksheet(worksheet_name)

# DataFrame をスプレッドシートに書き込む
#set_with_dataframe(worksheet, mansion_data)

In [15]:
import sqlite3

# SQLite データベースに接続
conn = sqlite3.connect('chuko_database.db')
cursor = conn.cursor()

# テーブル作成
cursor.execute('CREATE TABLE IF NOT EXISTS chuko_table (id INTEGER PRIMARY KEY, name TEXT, location_parts TEXT, price INTEGER, layout TEXT, area REAL, construction_year TEXT, transportation TEXT, minutes INTEGER, address TEXT, 区 TEXT)')

# データ挿入
for _, row in mansion_data.iterrows():
    cursor.execute('INSERT INTO chuko_table (name, location_parts, price, layout, area, construction_year, transportation, minutes, address, 区) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', 
                   (row['name'], row['location_parts'], row['price'], row['layout'], row['area'], row['construction_year'], row['transportation'], row['minutes'], row['address'], row['区']))

# コミットと接続のクローズ
conn.commit()
conn.close()

print("データベースにデータを保存しました。")

データベースにデータを保存しました。
